# Embedder

Embed chunked content with a high-recall tuned language model. For now, we're using the `HuggingFaceInstructEmbeddings`.

**Note**
- Run this notebook with an appropriate compute. 
- The code below installs the necessary requirements. 

In [1]:
!pip install sentence-transformers==2.2.2

In [2]:
!pip install InstructorEmbedding==1.0.1

In [2]:
!pip install neo4j

In [3]:
!pip install azure-identity azure-keyvault-secrets

# Get Credentials

In [4]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

key_vault_name = "kv-zojcqxmj"
kv_uri = f"https://{key_vault_name}.vault.azure.net/"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential)

# Now you can use neo4j_url, neo4j_port, and neo4j_password in your application
neo4j_url = client.get_secret("NEO4JURL").value
neo4j_user = client.get_secret("NEO4JUSER").value
neo4j_password = client.get_secret("NEO4JPASSWORD").value

In [5]:
import langchain
langchain.verbose = True
langchain.debug = True

In [6]:
import pandas as pd

# Read from pikle file
chunks_raw_df = pd.read_pickle('./data/chunks_raw.pkl')

chunks_raw_df.head()

,content,banner_title,banner_divisions,intro,toc,url,scrape_date,chunk_s500_o60,chunk_order
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Waarom willen de kinderartsen dat je baby m...,0
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Mogelijke afwijkingen\n\n\n### Aangeboren h...,1
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Infecties\n\n\nVerschillende infecties wor...,2
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Aangeboren darmafwijkingen\n\n\nHet is pas...,3
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,De kinderarts onderzoekt je baby in normale om...,4


## Load Embedding Model 

Hugginfacehub model

In [7]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl", 
    cache_folder='./models/model_cache_xl'
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
import pandas as pd
import numpy as np

def embed_str(s: str) -> np.ndarray:
    return np.float64(embeddings.embed_query(s))

def embed_df(df: pd.DataFrame, field:str) -> pd.DataFrame:
    df[f'{field}_embedding'] = (
        df[field].apply(embed_str)
    )
    return df

# Read from pikle file
chunks_processed_df = (
    chunks_raw_df.copy(deep=True)
    # Filter (testing)
    # .pipe(lambda df: df.head(100))
    .pipe(embed_df, field="chunk_s500_o60")
)

chunks_processed_df.to_pickle(path='./data/chunks_processed_full.pkl')

chunks_processed_df.head(15)

,content,banner_title,banner_divisions,intro,toc,url,scrape_date,chunk_s500_o60,chunk_order,chunk_s500_o60_embedding
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Waarom willen de kinderartsen dat je baby m...,0,"[0.04676021263003349, 0.009244767017662525, -0..."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Mogelijke afwijkingen\n\n\n### Aangeboren h...,1,"[0.0652947872877121, 0.012688170187175274, -0...."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Infecties\n\n\nVerschillende infecties wor...,2,"[0.09477945417165756, 0.012673679739236832, -0..."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Aangeboren darmafwijkingen\n\n\nHet is pas...,3,"[0.04747367277741432, 0.023175092414021492, -0..."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,De kinderarts onderzoekt je baby in normale om...,4,"[0.03207284212112427, 0.011688114143908024, -0..."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,* Observatie van het algemeen gedrag en drinkg...,5,"[0.06070693954825401, -0.013679267838597298, -..."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,* Meting van het bilirubinegehalte (= oorspron...,6,"[0.0663687065243721, -0.016540078446269035, 0...."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,Minimaal 48 uur observatie op de materniteit i...,7,"[0.057118576020002365, 0.01109850499778986, -0..."
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,AZ Sint-Lucas maakt gebruik van cookies om uw ...,8,"[0.04251086339354515, 0.03886432200670242, 0.0..."
1,\nWe weten dat reflux en opboeren voor een aan...,24 uur ph-metrie meting,[{'division_url': 'https://www.azstlucas.be/sp...,Een ph-metrie meting met impedantiemetin

In [36]:
# Save DF to Blob Storage for later retrieval. 
chunks_processed_df.to_pickle(path='./data/chunks_processed.pkl')